<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%202/2_3_Erster_Blick_in_die_Daten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Installieren aller Pakete

In [ ]:
# Hier die Kaggle Credentials einfügen (ohne Anführungszeichen)

%env KAGGLE_USERNAME=openhpi
%env KAGGLE_KEY=das_ist_der_key

In [ ]:
!pip install quickda
!pip install multidict

In [ ]:
import pandas as pd
from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize
import codecs
from wordcloud import WordCloud

import multidict as multidict

import numpy as np

import re
from PIL import Image
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib import rcParams
import string
rcParams['figure.figsize'] = 12, 8

%matplotlib inline

In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
pd.options.mode.chained_assignment = None

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
!pip3 install -r ../requirements.txt

In [ ]:
def getFrequencyDictForText(sentence, remove_stop_words=False):
 
    sentence = ' '.join(sentence)
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    if remove_stop_words:
        sentence_tokenized = word_tokenize(sentence)
        sentence = [word for word in sentence_tokenized if not word in stopwords.words()]
        sentence = ' '.join(sentence)
    fullTermsDict = multidict.MultiDict()
    tmpDict = {}

    for text in sentence.split(" "):
        val = tmpDict.get(text, 0)
        tmpDict[text.lower()] = val + 1
    for key in tmpDict:
        fullTermsDict.add(key, tmpDict[key])
    return fullTermsDict


def makeImage(text):

    wc = WordCloud(background_color="white", max_words=100)
    # generate word cloud
    wc.generate_from_frequencies(text)

    # show
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    
def get_film_descriptions(df):
    sentence_list = df.dropna().tolist()
    sentence_list_new = []
    for line in sentence_list:
        sentence_list_new.append(' '.join( [w for w in line.split() if len(w)>1] ))
    pattern = '[0-9]'
    sentence_list = [re.sub(pattern, '', str(i)) for i in sentence_list_new[:100]]
    return sentence_list

# 2.3 Erster Blick in die Daten

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/jakob-owens-CiUR8zISX60-unsplash%20(2).jpg">



Datensatz: 

### The Movies Dataset
Metadaten für über 45.000 Filme. 26 Millionen Bewertungen von über 270.000 NutzerInnen.

Quelle: kaggle.com

## Download Dataset 

### Manuell
via https://www.kaggle.com/rounakbanik/the-movies-dataset

### Via API

Hinzufügen der kaggle.json
Speichern als ~/.kaggle/kaggle.json auf Linux, OSX, oder andere UNIX-based Betriebssysteme und unter C:\Users<Windows-username>.kaggle\kaggle.json auf Windows

Siehe https://www.kaggle.com/docs/api oder https://github.com/Kaggle/kaggle-api
        
Beispiel:
~/kaggle/kaggle.json

{"username":"openHPI","key":"das_ist_der_key"}

In [ ]:
!pip3 install kaggle
!kaggle datasets download -d rounakbanik/the-movies-dataset

In [ ]:
import zipfile
with zipfile.ZipFile("the-movies-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("")

## Übersicht über alle Dateien

Zunächst sehen wir uns alle Dateien in diesem Datensatz einmal an. Wir werden für das Recommender-System hauptsächlich zwei Dateien näher ansehen: **movies_metadata.csv** und **ratings.csv**. Dafür wechseln wir zunächst in das Verzeichnis mit den Daten und lassen uns alle Dateien auflisten. Hier sieht man bereits die unterschiedliche Größe der verschiedenen Dateien.

In [ ]:
!ls -gnG

# Film Meta-Daten

Nun schauen wir genauer in die Film-Metadaten. Diese Datei werden wir hauptsächlich dazu verwenden, um die **content-based Recommendation** umzusetzen. Zunächst laden wir den Datensatz aus einer .csv-Datei. "csv" steht für comma-separated-values und ist ein sehr gängiges Datenformat neben z.B: .json im Machine Learning Bereich. 

In [ ]:
import pandas as pd
df_film_metadata = pd.read_csv("movies_metadata.csv", low_memory=False)

In [ ]:
df_film_metadata.head()

Damit wir einen besseren Überblick über alle Features gewinnen, nutzen wir eine Bibliothek, welche einfache Auswertungen bereits auf dem Datensatz und den Features für uns macht. 

In [ ]:
explore(df_film_metadata[df_film_metadata.columns.difference(['adult', "homepage", "id"])], method='profile', report_name='Report')

Sehen wir uns noch einmal übersichtlich alle Features an. Potentiell könnten wir hiervon mehrere Features für unsere content-based Recommendation heranziehen. Wir werden uns jedoch darauf fokussieren, Ähnlichkeit anhand der Filmbeschreibungen zu ermitteln. 

In [ ]:
df_film_metadata.columns.tolist()

Die Kurzbeschreibungen einzelner Filme findet sich in der Spalte **overview**. 

In [ ]:
word_list = get_film_descriptions(df_film_metadata["overview"])

Wir schauen uns als Beispiel einen ganz bekannten Film an. Auf Platz 10 in unserer Liste findet sich **Golden Eye** - ein Film aus der James Bond Filmreihe. Wir nutzen hier den Index 9, da auch in Python Listen im Index mit 0 beginnen. Will man also den zehnten Eintrag erhalten, wählt man Index 9. 

In [ ]:
print("Film-Titel: " + df_film_metadata["title"][9] + "\n")
print("Film-Beschreibung: " + word_list[9])

Um einen Überblick über die ganzen Filme in unserem Datensatz zu gewinnen, erstellen wir eine sogenannte Wort-Wolke/Word-Cloud, die alle Wörter in den Kurzbeschreibungen enthält. 

In [ ]:
makeImage(getFrequencyDictForText(word_list))

Hierbei fällt allerdings auf, dass sehr viele der Woche wenig Bedeutung haben für den Inhalt. Diese sogenannten **Stopwords** filtern wir nun aus den Beschreibungen der Filme und lassen uns erneut eine Wort-Wolke anzeigen.

In [ ]:
makeImage(getFrequencyDictForText(word_list, remove_stop_words=True))

# Film-Bewertungen der NutzerInnen


Nun betrachten wir auch die Film-Bewertungen für das Empfehlungssystem basierend auf **Collaborative Filtering**. Auch hier laden wir zunächst mal die jeweilige Date in ein Pandas DataFrame.

In [ ]:
df_film_votes = pd.read_csv("ratings.csv", low_memory=False)

Mit dem Befehl **.head()** können wir uns die ersten Einträge des DataFrames anzeigen lassen und bekommen ein Gefühl dafür, wie diese Datei und der Inhalt aufgebaut ist. 

In [ ]:
df_film_votes.head()

Wollen wir erfahren, wie viele einzelne NutzerInnen und Nutzer es gibt und wie viele einzelne Filme, so können wir das wie folgt machen. 

In [ ]:
df_film_votes[["movieId", "userId"]].nunique()

Da das von Bewertungsplattform zu Bewertungsplattform unterschiedlich gehandhabt wird, schauen uns wir die einzelnen Werte der Bewertungen einmal genauer an. Hier sehen wir, dass die Bewertungen in Schritten von 0,5 gegeben werden können von 0,5 bis 5,0. 

In [ ]:
sorted(df_film_votes["rating"].unique())

Um die Verteilung der Bewertungen einmal genauer zu sehen, geben wir uns ein sogenanntes Histogramm aus. Ein Histogramm zeigt die Häufigkeit einzelner Ausprägungen. Beispielsweise wie oft 3,0 als Bewertung gegeben wurde. 

In [ ]:
df_film_votes[["rating"]].hist()